<a href="https://colab.research.google.com/github/pfolaa/dsti-labs/blob/main/fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import glob

In [ ]:
#cd /content/drive/MyDrive/datasets/

/content/drive/MyDrive/datasets


## 1- Data acquisition

###1.1- Get files from repository 

In [131]:
#!wget https://{bucket-name}.s3.eu-west-1.amazonaws.com/{file.zip}

### 1.2 Extract zip file

In [2]:
from zipfile import ZipFile
#ZipFile("{file.zip}").extractall('/content/drive/My Drive/datasets/')

In [ ]:
os.

### 1.3 Read all json files and insert into a csv file

In [37]:
import os,json
import pandas as pd

# read json file
def read_Json_And_Insert_Into_CSV(path_file_json, file_csv, root_path):
  os.makedirs(root_path, exist_ok=True) # créer toute l'aborescence du fichier, crée le chemin
  # read all json files
  for file_name in [file for file in os.listdir(path_file_json) if file.endswith('.json')]:
    with open(path_file_json + file_name) as json_file:
      data = json.load(json_file)
      df = pd.DataFrame.from_records(data)

  # convert file to csv
  df.to_csv(f'{root_path}/{file_csv}', sep=';')
  return df # return du fichier csv

###NB: ne pas utiliser les memes noms de variables à l'intérieur des fonctions et à l'extérieur

In [38]:
path_json = '/content/drive/My Drive/datasets/nirra-log-bot/'
root_csv = '/content/drive/My Drive/datasets/nirra-log-bot/csv'
file_csv = 'file_name.csv'

df_raw = read_Json_And_Insert_Into_CSV(path_json, file_csv, root_csv)

In [39]:
df_raw.head()

,type,subtype,text,ts,bot_id
0,message,bot_message,"[info] - [""[OKRA WEBHOOK]:"",""{\""method\"":\""PAY...",1630911747.008800,B01RC3AKYG6
1,message,bot_message,"[info] - [""[API REQUEST]: 08134676606, /client...",1630911747.008900,B01RC3AKYG6
2,message,bot_message,"[info] - [""[API REQUEST]: 08134676606, /client...",1630911749.009000,B01RC3AKYG6
3,message,bot_message,"[info] - [""[API REQUEST]: 08134676606, /client...",1630911754.009100,B01RC3AKYG6
4,message,bot_message,"[info] - [""[API REQUEST]: 08134676606, /client...",1630911754.009200,B01RC3AKYG6


### OLD: Get total number of @ inside "text" column

In [40]:
 df_raw['text'].str.contains("@").sum() # search total number of @ within text column

37

In [41]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4953 entries, 0 to 4952
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   type     4953 non-null   object
 1   subtype  4953 non-null   object
 2   text     4953 non-null   object
 3   ts       4953 non-null   object
 4   bot_id   4953 non-null   object
dtypes: object(5)
memory usage: 193.6+ KB


### OLD: Test a slicing

In [25]:
# slicing
#df_raw[df_raw['text'].str.contains("@")] # masque à l'intérieur des crochets

In [10]:
df_raw[df_raw['text'] == None]

,type,subtype,text,ts,bot_id


### OLD: Used in case of OKRA WEBHOOK, WALLET SUCCESS, SMS SUCCESS, SMS PAYLOAD Types request

In [42]:
import regex
import json

#text_okra_webhook = df_raw['text'][0]
#text_wallet = df_raw['text'][841]
#sms_success = df_raw['text'][12]
#sms_payload = df_raw['text'][11]
def parse_wallet_sms_payload_success(text_type_request):
  ''' la fonction permet de parser les types de requete "Okra WebHook", "Wallet success", 
      "SMS Success" et SMS Payload en object json.
      Elle prend en paramètre le text contenu dans le type de requete,
      elle retourne un objet de type JSON.'''

  pattern = regex.compile(r'\{(?:[^{}]|(?R))*}')
  resul_patt = pattern.findall(text_type_request)
  res = resul_patt[0].replace("\\", " ")
  s = json.loads(res)
  out_dict = {} # dictionnary vide
  for key, value in s.items():
    out_dict[key.strip()] = value # à la clé on passe chaque valeur, strip() enlève les espaces au début et à la fin.


  out_dump = json.dumps(out_dict) # input est un dictionnaire et ça retourne un json sous forme string
  out_wallet_success = json.loads(out_dump) # convertir le string json en object json.
  return out_wallet_success



In [43]:
# Test Okra WebHook
okra = parse_wallet_sms_payload_success(df_raw['text'][0])
print(okra)
print(okra.get('accountId'))
print(okra.get('authorization'))
print(okra.get('authorization').get('_id '))
print(okra.get('authorization').get('customer '))
print(okra.get('authorization').get('env '))
print(okra.get('authorization').get('owner '))

{'method': 'PAYMENT_AUTHORIZATION ', 'callback_type': 'PAYMENT_AUTHORIZATION ', 'callback_code': 'PAYMENT_AUTHORIZATION_SUCCESS ', 'type': 'CALLBACK ', 'code': 'PAYMENT_AUTHORIZATION_SUCCESS ', 'callbackURL': '<https://liberta.finsoftapp.com/okra/notification/push >', 'env': 'production ', 'status': 'is_success ', 'started_at': '2021-09-06T07:02:17.408Z ', 'ended_at': '2021-09-06T07:02:26.523Z ', 'message': 'Record callback started! ', 'accountId': '5f879cdbaa47f26f4750f63c ', 'options': {'loan_id ': '2598 '}, 'meta': {'loan_id ': '2598 '}, 'bankName': 'Guaranty Trust Bank ', 'bankType': 'ind ', 'bankId': '5d6fe57a4099cc4b210bbeb3 ', 'bankSlug': 'guaranty-trust-bank ', 'record': '6135bce91c51570c3e6a834d ', 'recordId': '6135bce91c51570c3e6a834d ', 'callback_url': '<https://api.okra.ng/v2/callback?record=6135bce91c51570c3e6a834d&amp;method=PAYMENT_AUTHORIZATION >', 'login_type': 'mobile ', 'customerId': '5f879cdbaa47f26f4750f645 ', 'identityType': 'bvn ', 'customerEmail': [' '], 'countr

In [44]:
# Test Wallet Success
wallet_success = parse_wallet_sms_payload_success(df_raw['text'][841])
print(wallet_success.get('account_name'))
print(wallet_success.get('account_number'))
print(wallet_success.get('bvn'))
print(wallet_success.get('requestSuccessful'))
print(wallet_success.get('responseCode'))
print(wallet_success.get('responseMessage'))


LIBERTA(Kayode  Ajao) 
9980432021 
22213975706 
True
00 
Reserved Account Generated Successfully 


In [45]:
# Test SMS Sucess
sms_succ = parse_wallet_sms_payload_success(df_raw['text'][12])
sms_succ.get('response').get('cost ')
sms_succ.get('response').get('status ')
sms_succ.get('response').get('totalsent ')
sms_succ.get('response')

{'cost ': 2, 'status ': 'SUCCESS ', 'totalsent ': 1}

In [46]:
# Test SMS Payload
sms_payload = parse_wallet_sms_payload_success(df_raw['text'][11])
print(sms_payload)
print(sms_payload.get('message'))
print(sms_payload.get('phone'))

{'phone': '2347035864882 ', 'message': 'To continue your signup, use this OTP 169 600 '}
To continue your signup, use this OTP 169 600 
2347035864882 


# OLD: Handle row with type request "LEADWAY SUCCESS" and concatenate rows
### NB: faire un docstring (''' ''') pour chaque fonction

In [47]:
import re

# la fonction doit prendre en paramètre quelque chose
def parse_and_concatenate_Leadway_Success_Rows(df_raw):
  '''Cette fonction permet de parser et de concatener le texte qui a LEADWAY SUCCESS
     comme type de requete
     elle prend comme paramètre un dataframe et retourne les valeurs suivantes:
     - un texte concatené
     - l'index de la 1ère ligne qu'on va utiliser ensuite pour l'effacer
     - l'index de la dernière ligne qu'on va utiliser ensuite pour l'effacer '''

  first_index = 0
  last_index = 0
  text_leadway_concat = ''
  for index, row in df_raw.iterrows():  # boucler sur les colonnes de type text
      text_row = row['text']  
      if re.search('LEADWAY SUCCESS', text_row):
        text_leadway_concat = text_row
        first_index = index
        first_index +=1
        new_df = df_raw[first_index:]
        for first_index, new_row in new_df.iterrows():
          xxx = new_row['text']      
          if not xxx.startswith('['):          
            first_index += 1
            text_leadway_concat = text_row + xxx       
          elif xxx.startswith('['):
            last_index = first_index-1
            break


  return text_leadway_concat, first_index, last_index


In [48]:
leadway_succ, first_index_succ, last_index_succ = parse_and_concatenate_Leadway_Success_Rows(df_raw)
leadway_succ, first_index_succ, last_index_succ

('[info] - ["[LEADWAY SUCCESS]:","{\\"errorMsg\\":\\"\\",\\"signature\\":\\"\\",\\"success\\":true,\\"vehicleMake\\":[{\\"id\\":\\"127\\",\\"makeName\\":\\"35CMB\\"},{\\"id\\":\\"152\\",\\"makeName\\":\\"AC ATUL\\"},{\\"id\\":\\"228\\",\\"makeName\\":\\"ACE\\"},{\\"id\\":\\"147\\",\\"makeName\\":\\"ACURA\\"},{\\"id\\":\\"8\\",\\"makeName\\":\\"ALPHA ROMEO\\"},{\\"id\\":\\"106\\",\\"makeName\\":\\"APACHE\\"},{\\"id\\":\\"210\\",\\"makeName\\":\\"ASHOK LEYLAND\\"},{\\"id\\":\\"80\\",\\"makeName\\":\\"ASTRA\\"},{\\"id\\":\\"3\\",\\"makeName\\":\\"AUDI\\"},{\\"id\\":\\"128\\",\\"makeName\\":\\"AUSTIN\\"},{\\"id\\":\\"109\\",\\"makeName\\":\\"BACK HOE\\"},{\\"id\\":\\"9\\",\\"makeName\\":\\"BADFORD\\"},{\\"id\\":\\"68\\",\\"makeName\\":\\"BAJAJ\\"},{\\"id\\":\\"218\\",\\"makeName\\":\\"BASUKI\\"},{\\"id\\":\\"248\\",\\"makeName\\":\\"BAW\\"},{\\"id\\":\\"244\\",\\"makeName\\":\\"BEDFORD\\"},{\\"id\\":\\"84\\",\\"makeName\\":\\"BEIBEN\\"},{\\"id\\":\\"195\\",\\"makeName\\":\\"BELL\\"},{\\"id

### OLD: Parse row Leadway Success to json

In [49]:
# use this function when type request is LEADWAY SUCCESS
import regex
import json


def parse_Leadway_Success_Row(text_leadway):
  ''' fonction permettant de parser le text concatené pour le type de requet LEADWAY SUCCESS
      elle retourner un dictionnaire.'''
  pattern = regex.compile(r'\{(?:[^{}]|(?R))*}')
  resul_patt = pattern.findall(text_leadway)
  resul_patt[0] = resul_patt[0].replace("\\", "")
  x = resul_patt[0].replace("make,", "")
  y = x.replace('""makeName"', '"makeName"')
  z = json.loads(y)
  vehicleMake = z.get('vehicleMake')
  leadway_dict = {}
  for element in vehicleMake:
    leadway_dict[element['id']] = element['makeName']

  return leadway_dict



In [50]:
print(leadway_succ)

[info] - ["[LEADWAY SUCCESS]:","{\"errorMsg\":\"\",\"signature\":\"\",\"success\":true,\"vehicleMake\":[{\"id\":\"127\",\"makeName\":\"35CMB\"},{\"id\":\"152\",\"makeName\":\"AC ATUL\"},{\"id\":\"228\",\"makeName\":\"ACE\"},{\"id\":\"147\",\"makeName\":\"ACURA\"},{\"id\":\"8\",\"makeName\":\"ALPHA ROMEO\"},{\"id\":\"106\",\"makeName\":\"APACHE\"},{\"id\":\"210\",\"makeName\":\"ASHOK LEYLAND\"},{\"id\":\"80\",\"makeName\":\"ASTRA\"},{\"id\":\"3\",\"makeName\":\"AUDI\"},{\"id\":\"128\",\"makeName\":\"AUSTIN\"},{\"id\":\"109\",\"makeName\":\"BACK HOE\"},{\"id\":\"9\",\"makeName\":\"BADFORD\"},{\"id\":\"68\",\"makeName\":\"BAJAJ\"},{\"id\":\"218\",\"makeName\":\"BASUKI\"},{\"id\":\"248\",\"makeName\":\"BAW\"},{\"id\":\"244\",\"makeName\":\"BEDFORD\"},{\"id\":\"84\",\"makeName\":\"BEIBEN\"},{\"id\":\"195\",\"makeName\":\"BELL\"},{\"id\":\"174\",\"makeName\":\"BENTLEY\"},{\"id\":\"75\",\"makeName\":\"BHACHU\"},{\"id\":\"262\",\"makeName\":\"BIKEMASTER\"},{\"id\":\"92\",\"makeName\":\"BLUE BI

In [ ]:
# TEST LEADWAY SUCCESS
resultat_leadway = parse_Leadway_Success_Row(leadway_succ)

for i in resultat_leadway:
  print(resultat_leadway.get(i))



### OLD: Drop concatenated rows! (faire une méthode qui marcherait si on a plusieurs lignes à effacer)

In [19]:
df_raw = df_raw.drop(first_index_succ)
df_raw = df_raw.drop(last_index_succ)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4951 entries, 0 to 4952
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   type     4951 non-null   object
 1   subtype  4951 non-null   object
 2   text     4951 non-null   object
 3   ts       4951 non-null   object
 4   bot_id   4951 non-null   object
dtypes: object(5)
memory usage: 232.1+ KB


In [67]:
2356 in df_raw.index

True

### OLD: Check rows that do not start with [ and drop them
NB: faire un docstring (''' ''') pour chaque fonction

In [20]:
 df_sans_crochet = []
 for index, row in df_raw.iterrows():
    str_text_1 = row['text']
    if not str_text_1.startswith('['):
      df_sans_crochet.append(row)    

In [21]:
pd.DataFrame(df_sans_crochet)

,type,subtype,text,ts,bot_id
2356,message,bot_message,SZxKLKSMVt2OavYly1g5M3tjyWWwsjs9cfI7xg5LY/XJZN...,1630951981.029800,B01RC3AKYG6
2358,message,bot_message,cZLwYJdmiKcLEoSjK0Xxy9R0QivBncaw+DlxTKlL0ttPZc...,1630951981.030000,B01RC3AKYG6
2359,message,bot_message,Y9kUk0UT2OwQ0USNPBDVdbDoS98jAaxgPgQBTCw0hfkId4...,1630951981.030100,B01RC3AKYG6
3059,message,bot_message,"Name\"":\""GRAND CHEROKEE\""},{\""id\"":\""115\"",\""m...",1630963547.006200,B01RC3AKYG6


### OLD: Handle log level "Error"
NB: faire un docstring (''' ''') pour chaque fonction

In [51]:
import re
df_raw['text'][2355]

text_error = df_raw['text'][2355].replace('"', "'")
pattern = regex.compile(r"{?[a-z :A-Z 0-9\\,=_`']+selfie")
resul_patt = pattern.findall(text_error)
ress = resul_patt[0].replace("\\", " ").replace("'name'", "name")
a = re.sub("`", "", ress)
a

"{ 'code ': 'ER_BAD_NULL_ERROR ', 'errno ':1048, 'sqlMessage ': 'Column name cannot be null ', 'sqlState ': '23000 ', 'index ':0, 'sql ': 'INSERT INTO preapplications Set product = 11, loan_amount = 5000, tenor = 1, loan_purpose = '18', tenor_type = 'monthly', selfie"

In [52]:
import regex
import json

def parse_Error_Row(error_row):
  error_row = error_row.replace('"', "'")
  pattern = regex.compile(r"{?[a-z :A-Z 0-9\\,=_`']+selfie")
  resul_patt = pattern.findall(error_row)
  res = resul_patt[0].replace("\\", " ")
  res = res.replace("'name'", "name").replace("`", "").replace("'18'", "18").replace("'monthly'", "monthly")
  res = res+'"}'
  res = res.replace("'", '"')
  s = json.loads(res)
  out_error_dict = {} # dictionnary vide
  for key, value in s.items():
    out_error_dict[key.strip()] = value # à la clé on passe chaque valeur, strip() enlève les espaces au début et à la fin.

  out_error_dump = json.dumps(out_error_dict) # input est un dictionnaire et ça retourne un json sous forme string
  out_error_text = json.loads(out_error_dump) # convertir le string json en object json.
  return out_error_text


In [53]:
error_text = parse_Error_Row(df_raw['text'][2355])
print(error_text)
print(error_text.get('code'))
print(error_text.get('errno'))
print(error_text.get('sqlMessage'))
print(error_text.get('sqlState'))
print(error_text.get('index'))
print(error_text.get('sql'))

{'code': 'ER_BAD_NULL_ERROR ', 'errno': 1048, 'sqlMessage': 'Column name cannot be null ', 'sqlState': '23000 ', 'index': 0, 'sql': 'INSERT INTO preapplications Set product = 11, loan_amount = 5000, tenor = 1, loan_purpose = 18, tenor_type = monthly, selfie'}
ER_BAD_NULL_ERROR 
1048
Column name cannot be null 
23000 
0
INSERT INTO preapplications Set product = 11, loan_amount = 5000, tenor = 1, loan_purpose = 18, tenor_type = monthly, selfie


### OLD: Function to convert data to timestamp

In [54]:
import datetime

# function to convert date to Timestamp
def convertToTimestamp(str):
  element = datetime.datetime.strptime(str,"%Y-%m-%dT%H:%M:%S.%fZ")
  return datetime.datetime.timestamp(element)

In [25]:
timestamp = convertToTimestamp('2021-09-05T07:03:55.223Z')
timestamp

1630825435.223

## 2 Data model
### 2.1 Parse rows of dataset

### 2.1.1 créer un dictionnary pour les regex.

In [55]:
type_request_dictionnary = {}
regex_list_api_request = []
regex_list_api_request.append('[\w.+-]+@[\w-]+\.[\w.-]+')
regex_list_api_request.append('/[/a-z 0-9?=&;/_A-Z+]+')
regex_list_api_request.append('(\d{4})-(\d\d)-(\d\d)T(\d\d):(\d\d):(\d\d).(\d{3})*[a-zA-Z]')
regex_list_api_request.append('[0-9]+')

type_request_dictionnary['API REQUEST'] = regex_list_api_request

regex_list_client_mobile = []
regex_list_client_mobile.append('[\w.+-]+@[\w-]+\.[\w.-]+')
regex_list_client_mobile.append('(\d{4})-(\d\d)-(\d\d)T(\d\d):(\d\d):(\d\d).(\d{3})*[a-zA-Z]')
regex_list_client_mobile.append('[0-9]+')

type_request_dictionnary['CLIENT MOBILE LOGIN'] = regex_list_client_mobile


type_request_dictionnary['SMS PAYLOAD'] = '\{(?:[^{}]|(?R))*}'
type_request_dictionnary['SMS SUCCESS'] = '\{(?:[^{}]|(?R))*}'
type_request_dictionnary['WALLET SUCCESS'] = '\{(?:[^{}]|(?R))*}'
type_request_dictionnary['LEADWAY SUCCESS'] = '\{(?:[^{}]|(?R))*}'



### 2.1.2 Fonction. 

In [28]:
import re
import regex
import json

# parse row of text column

def parse_row_error(df_):
  log_level_col = []
  api_request_col = []
  type_request_col = []
  phone_Col = []
  date_col = []
  endpoint_Col = []
  email_col = []
  message_sms_payload_col = []
  totalsent_col = []
  cost_col = []
  status_col = []
  account_number_col = []
  account_name_col = []
  bvn_col = []
  requestSuccessful_col = []
  responseMessage_col = []
  responseCode_col = []
  error_code_col = []
  error_number_col = []
  error_sql_message_col = []
  error_sql_state_col = []
  error_index_col = []
  error_sql_col = []


  list_column_none_api_request = []
  list_column_none_api_request.append(message_sms_payload_col)
  list_column_none_api_request.append(totalsent_col)
  list_column_none_api_request.append(cost_col)
  list_column_none_api_request.append(status_col)
  list_column_none_api_request.append(account_number_col)
  list_column_none_api_request.append(bvn_col)
  list_column_none_api_request.append(requestSuccessful_col)
  list_column_none_api_request.append(responseMessage_col)
  list_column_none_api_request.append(responseCode_col)
  list_column_none_api_request.append(account_name_col)
  list_column_none_api_request.append(error_code_col)
  list_column_none_api_request.append(error_number_col)
  list_column_none_api_request.append(error_sql_message_col)
  list_column_none_api_request.append(error_sql_state_col)
  list_column_none_api_request.append(error_index_col)
  list_column_none_api_request.append(error_sql_col)
  
  list_column_none_client_mobile = []
  list_column_none_client_mobile.append(message_sms_payload_col)
  list_column_none_client_mobile.append(totalsent_col)
  list_column_none_client_mobile.append(cost_col)
  list_column_none_client_mobile.append(status_col)
  list_column_none_client_mobile.append(account_number_col)
  list_column_none_client_mobile.append(bvn_col)
  list_column_none_client_mobile.append(requestSuccessful_col)
  list_column_none_client_mobile.append(responseMessage_col)
  list_column_none_client_mobile.append(responseCode_col)
  list_column_none_client_mobile.append(account_name_col)
  list_column_none_client_mobile.append(endpoint_Col)
  list_column_none_client_mobile.append(error_code_col)
  list_column_none_client_mobile.append(error_number_col)
  list_column_none_client_mobile.append(error_sql_message_col)
  list_column_none_client_mobile.append(error_sql_state_col)
  list_column_none_client_mobile.append(error_index_col)
  list_column_none_client_mobile.append(error_sql_col)

  list_column_none_sms_payload = []
  list_column_none_sms_payload.append(totalsent_col)
  list_column_none_sms_payload.append(cost_col)
  list_column_none_sms_payload.append(status_col)
  list_column_none_sms_payload.append(account_number_col)
  list_column_none_sms_payload.append(bvn_col)
  list_column_none_sms_payload.append(requestSuccessful_col)
  list_column_none_sms_payload.append(responseMessage_col)
  list_column_none_sms_payload.append(responseCode_col)
  list_column_none_sms_payload.append(account_name_col)
  list_column_none_sms_payload.append(email_col)
  list_column_none_sms_payload.append(endpoint_Col)
  list_column_none_sms_payload.append(date_col)
  list_column_none_sms_payload.append(error_code_col)
  list_column_none_sms_payload.append(error_number_col)
  list_column_none_sms_payload.append(error_sql_message_col)
  list_column_none_sms_payload.append(error_sql_state_col)
  list_column_none_sms_payload.append(error_index_col)
  list_column_none_sms_payload.append(error_sql_col)

  list_column_none_sms_success = []
  list_column_none_sms_success.append(message_sms_payload_col)
  list_column_none_sms_success.append(account_number_col)
  list_column_none_sms_success.append(bvn_col)
  list_column_none_sms_success.append(requestSuccessful_col)
  list_column_none_sms_success.append(responseMessage_col)
  list_column_none_sms_success.append(responseCode_col)
  list_column_none_sms_success.append(account_name_col)
  list_column_none_sms_success.append(email_col)
  list_column_none_sms_success.append(phone_Col)
  list_column_none_sms_success.append(endpoint_Col)
  list_column_none_sms_success.append(date_col)
  list_column_none_sms_success.append(error_code_col)
  list_column_none_sms_success.append(error_number_col)
  list_column_none_sms_success.append(error_sql_message_col)
  list_column_none_sms_success.append(error_sql_state_col)
  list_column_none_sms_success.append(error_index_col)
  list_column_none_sms_success.append(error_sql_col)

  list_column_none_wallet_success = []
  list_column_none_wallet_success.append(totalsent_col)
  list_column_none_wallet_success.append(message_sms_payload_col)
  list_column_none_wallet_success.append(cost_col)
  list_column_none_wallet_success.append(status_col)
  list_column_none_wallet_success.append(email_col)
  list_column_none_wallet_success.append(phone_Col)
  list_column_none_wallet_success.append(endpoint_Col)
  list_column_none_wallet_success.append(date_col)
  list_column_none_wallet_success.append(error_code_col)
  list_column_none_wallet_success.append(error_number_col)
  list_column_none_wallet_success.append(error_sql_message_col)
  list_column_none_wallet_success.append(error_sql_state_col)
  list_column_none_wallet_success.append(error_index_col)
  list_column_none_wallet_success.append(error_sql_col)

  list_column_none_leadway_success = []
  list_column_none_leadway_success.append(message_sms_payload_col)
  list_column_none_leadway_success.append(totalsent_col)
  list_column_none_leadway_success.append(cost_col)
  list_column_none_leadway_success.append(status_col)
  list_column_none_leadway_success.append(account_number_col)
  list_column_none_leadway_success.append(bvn_col)
  list_column_none_leadway_success.append(requestSuccessful_col)
  list_column_none_leadway_success.append(responseMessage_col)
  list_column_none_leadway_success.append(responseCode_col)
  list_column_none_leadway_success.append(account_name_col)
  list_column_none_leadway_success.append(email_col)
  list_column_none_leadway_success.append(phone_Col)
  list_column_none_leadway_success.append(endpoint_Col)
  list_column_none_leadway_success.append(date_col)
  list_column_none_leadway_success.append(error_code_col)
  list_column_none_leadway_success.append(error_number_col)
  list_column_none_leadway_success.append(error_sql_message_col)
  list_column_none_leadway_success.append(error_sql_state_col)
  list_column_none_leadway_success.append(error_index_col)
  list_column_none_leadway_success.append(error_sql_col)
  
  list_column_none_level_log_error = []
  list_column_none_level_log_error.append(message_sms_payload_col)
  list_column_none_level_log_error.append(totalsent_col)
  list_column_none_level_log_error.append(cost_col)
  list_column_none_level_log_error.append(status_col)
  list_column_none_level_log_error.append(email_col)
  list_column_none_level_log_error.append(phone_Col)
  list_column_none_level_log_error.append(endpoint_Col)
  list_column_none_level_log_error.append(date_col)
  list_column_none_level_log_error.append(bvn_col)
  list_column_none_level_log_error.append(requestSuccessful_col)
  list_column_none_level_log_error.append(responseMessage_col)
  list_column_none_level_log_error.append(responseCode_col)
  list_column_none_level_log_error.append(account_name_col)
  list_column_none_level_log_error.append(account_number_col)
  
  list_all_colum = []
  list_all_colum.append(type_request_col)
  list_all_colum.append(phone_Col)
  list_all_colum.append(date_col)
  list_all_colum.append(endpoint_Col)
  list_all_colum.append(log_level_col)
  list_all_colum.append(email_col)
  list_all_colum.append(message_sms_payload_col)
  list_all_colum.append(totalsent_col)
  list_all_colum.append(cost_col)
  list_all_colum.append(status_col)
  list_all_colum.append(account_number_col)
  list_all_colum.append(account_name_col)
  list_all_colum.append(bvn_col)
  list_all_colum.append(requestSuccessful_col)
  list_all_colum.append(responseMessage_col)
  list_all_colum.append(responseCode_col)
  list_all_colum.append(error_code_col)
  list_all_colum.append(error_number_col)
  list_all_colum.append(error_sql_message_col)
  list_all_colum.append(error_sql_state_col)
  list_all_colum.append(error_index_col)
  list_all_colum.append(error_sql_col)
  

  for index, row in df_.iterrows():
    str_text = row['text']
    
    if not str_text.startswith('['):
      for i in range(len(list_all_colum)):
          list_all_colum[i].append(None)

    # check if the row contains "info" string
    if re.search('info', str_text):
        log_level = re.search('info', str_text)
        try:
          log_level_col.append(log_level.group(0))
        except AttributeError:
          log_level_col.append(None)        
        # check if the row contains an email address 
        # pour tous les types request créer un dictionnaire dans lequel mapper
        # key = type de request et value = les regex définis
        # pour chaque condition IF créer une liste de colonnes auxquelles affecter None
        if 'mailto' in str_text:
            if re.search('API REQUEST', str_text):
                type_of_request = re.search('API REQUEST', str_text)                
                phone_or_email_api_req = re.search(type_request_dictionnary['API REQUEST'][0], str_text)                              
                endpoint = re.search(type_request_dictionnary['API REQUEST'][1], str_text)
                pattern = type_request_dictionnary['API REQUEST'][2]
                datepattern = re.compile("(?:%s)"%(pattern))
                datematcher = datepattern.search(str_text)  # extract date

                for i in range(len(list_column_none_api_request)):
                  list_column_none_api_request[i].append(None)
                
                #print('column none api request:')
                #print(list_column_none_api_request)
                               
                try:
                  type_request_col.append(type_of_request.group(0)) # add type request inside type request column
                except AttributeError:
                  type_request_col.append(None)               
                try:
                  email_col.append(phone_or_email_api_req.group(0)) # add email inside email column
                  phone_Col.append(None)  # in this case there is no phone number
                except AttributeError:
                  email_col.append(None)
                try:
                  endpoint_Col.append(endpoint.group(0)) # add endpoint inside endpoint column
                except AttributeError:
                  endpoint_Col.append(None)
                try:
                  date_col.append(convertToTimestamp(datematcher.group(0))) # convert date to timestamp and add it inside date column
                except AttributeError:
                  date_col.append(None)
              
            elif re.search('CLIENT MOBILE LOGIN', str_text):   # CLIENT MOBILE LOGIN with email address
                  type_of_request = re.search('CLIENT MOBILE LOGIN', str_text)
                  # extract address email for CLIENT MOBILE LOGIN
                  phone_or_email_client_mobile = re.search(type_request_dictionnary['CLIENT MOBILE LOGIN'][0], str_text)                               
                  pattern = type_request_dictionnary['CLIENT MOBILE LOGIN'][1]
                  datepattern = re.compile("(?:%s)"%(pattern))
                  datematcher = datepattern.search(str_text)  # extract date for CLIENT MOBILE LOGIN type request
                  
                  for j in range(len(list_column_none_client_mobile)):
                    list_column_none_client_mobile[j].append(None)

                  #print('column none client mobile:')
                  #print(list_column_none_client_mobile)
                  try:
                    type_request_col.append(type_of_request.group(0)) # add type request inside type request column
                  except AttributeError:
                    type_request_col.append(None) 
                  try:
                    email_col.append(phone_or_email_client_mobile.group(0)) # add email inside email column
                    phone_Col.append(None)  # in this case there is no phone number
                  except AttributeError:
                    email_col.append(None)
                  try:
                    date_col.append(convertToTimestamp(datematcher.group(0))) # convert date to timestamp and add it inside date column
                  except AttributeError:
                    date_col.append(None)

        elif 'mailto' not in str_text:
            if re.search('API REQUEST', str_text):
                type_of_request = re.search('API REQUEST', str_text)                
                endpoint = re.search('/[/a-z 0-9?=&;/_A-Z+]+', str_text) # extract endpoint             
                # extract a phone number for API REQUEST
                phone_or_email_api_req = re.search(type_request_dictionnary['API REQUEST'][3], str_text)                              
                endpoint = re.search(type_request_dictionnary['API REQUEST'][1], str_text)
                pattern = type_request_dictionnary['API REQUEST'][2]
                datepattern = re.compile("(?:%s)"%(pattern))
                datematcher = datepattern.search(str_text)  # extract date

                for i in range(len(list_column_none_api_request)):
                  list_column_none_api_request[i].append(None)
                
                #print('column none api request no mail:')
                #print(list_column_none_api_request)
                try:
                  phone_Col.append(phone_or_email_api_req.group(0)) # add phone number inside phone number column
                  email_col.append(None) # in this case there is no email address
                except AttributeError:
                  phone_Col.append(None)
                try:
                  type_request_col.append(type_of_request.group(0))
                except AttributeError:
                  type_request_col.append(None)
                try:
                  endpoint_Col.append(endpoint.group(0)) # add endpoint inside endpoint column
                except AttributeError:
                  endpoint_Col.append(None)
                try:
                  date_col.append(convertToTimestamp(datematcher.group(0))) # convert date to timestamp and add it inside date column
                except AttributeError:
                  date_col.append(None)
            elif re.search('CLIENT MOBILE LOGIN', str_text): # when type request is CLIENT MOBILE LOGIN, there is no EndPoint
                type_of_request = re.search('CLIENT MOBILE LOGIN', str_text)
                # extract a phone number for CLIENT MOBILE LOGIN
                phone_or_email_client_mobile = re.search(type_request_dictionnary['CLIENT MOBILE LOGIN'][2], str_text)                  
                pattern = type_request_dictionnary['CLIENT MOBILE LOGIN'][1]
                datepattern = re.compile("(?:%s)"%(pattern))
                datematcher = datepattern.search(str_text)  # extract date

                for j in range(len(list_column_none_client_mobile)):
                    list_column_none_client_mobile[j].append(None)

                #print('column none client mobile no mail:')
                #print(list_column_none_client_mobile)
                try:
                  phone_Col.append(phone_or_email_client_mobile.group(0))
                  email_col.append(None)
                except AttributeError:
                  phone_Col.append(None)
                try:
                  type_request_col.append(type_of_request.group(0))
                except AttributeError:
                  type_request_col.append(None)
                try:
                  date_col.append(convertToTimestamp(datematcher.group(0))) # convert date to timestamp and add it inside date column
                except AttributeError:
                  date_col.append(None)

            elif re.search('SMS PAYLOAD', str_text):
                type_of_request = re.search('SMS PAYLOAD', str_text)            
                sms_payload = parse_wallet_sms_payload_success(str_text)               
                for l in range(len(list_column_none_sms_payload)):
                    list_column_none_sms_payload[l].append(None)             

                #print('column none sms payload:')
                #print(list_column_none_sms_payload)
                try:
                  type_request_col.append(type_of_request.group(0))
                except AttributeError:
                  type_request_col.append(None)
                try:
                  phone_Col.append(sms_payload.get('phone'))
                except AttributeError:
                  phone_Col.append(None)
                try:
                  message_sms_payload_col.append(sms_payload.get('message'))
                except AttributeError:
                  message_sms_payload_col.append(None)
                
            elif re.search('SMS SUCCESS', str_text): 
                type_of_request = re.search('SMS SUCCESS', str_text)
                sms_success = parse_wallet_sms_payload_success(str_text)                
                for m in range(len(list_column_none_sms_success)):
                    list_column_none_sms_success[m].append(None) 
                
                #print('column none sms success:')
                #print(list_column_none_sms_success)             
                try:
                  type_request_col.append(type_of_request.group(0))
                except AttributeError:
                  type_request_col.append(None)
                try:                 
                  totalsent_col.append(sms_success.get('response').get('totalsent '))
                except AttributeError:
                  totalsent_col.append(None)
                try:                 
                  cost_col.append(sms_success.get('response').get('cost '))
                except AttributeError:
                  cost_col.append(None)
                try:                 
                  status_col.append(sms_success.get('response').get('status '))
                except AttributeError:
                  status_col.append(None)
            elif re.search('WALLET PAYLOAD', str_text):
                  type_of_request = re.search('WALLET PAYLOAD', str_text)
                  message_sms_payload_col.append(None)
                  totalsent_col.append(None)
                  cost_col.append(None)
                  status_col.append(None)
                  account_number_col.append(None)
                  bvn_col.append(None)
                  requestSuccessful_col.append(None)
                  responseMessage_col.append(None)
                  responseCode_col.append(None)
                  account_name_col.append(None)
                  email_col.append(None)
                  phone_Col.append(None)
                  endpoint_Col.append(None)
                  date_col.append(None)
                  try:
                    type_request_col.append(type_of_request.group(0))
                  except AttributeError:
                    type_request_col.append(None)
            elif re.search('WALLET SUCCESS', str_text):
                  wallet_success = parse_wallet_sms_payload_success(str_text)

                  try:
                    type_request_col.append(type_of_request.group(0))
                  except AttributeError:
                    type_request_col.append(None)
                  try:
                    account_number_col.append(wallet_success.get('account_number'))
                  except AttributeError:
                    account_number_col.append(None)
                  try:
                    account_name_col.append(wallet_success.get('account_name'))
                  except AttributeError:
                    account_name_col.append(None)
                  try:
                    bvn_col.append(wallet_success.get('bvn'))
                  except AttributeError:
                    bvn_col.append(None)
                  try:
                    requestSuccessful_col.append(wallet_success.get('requestSuccessful'))
                  except AttributeError:
                    requestSuccessful_col.append(None)
                  try:
                    responseMessage_col.append((wallet_success.get('responseMessage')))
                  except AttributeError:
                    responseMessage_col.append(None)
                  try:
                    responseCode_col.append((wallet_success.get('responseCode')))
                  except AttributeError:
                    responseCode_col.append(None)
                  for n in range(len(list_column_none_wallet_success)):
                    list_column_none_wallet_success[n].append(None) 

                  #print('column none wallet success:')
                  #print(list_column_none_wallet_success)
                  
            elif re.search('OKRA WEBHOOK', str_text):
                  type_of_request = re.search('OKRA WEBHOOK', str_text)
                  okra_webhook= parse_wallet_sms_payload_success(str_text)
                  try:
                    type_request_col.append(type_of_request.group(0))
                  except AttributeError:
                    type_request_col.append(None)
                  
                  for n in range(len(list_column_none_wallet_success)):
                    list_column_none_wallet_success[n].append(None)
                  
                  #print('column none okra webhook success:')
                  #print(list_column_none_wallet_success)

            elif re.search('LEADWAY SUCCESS', str_text):
                  type_of_request = re.search('LEADWAY SUCCESS', str_text)
                  leadway_success_concat_text, index_first_succ, index_last_succ = parse_and_concatenate_Leadway_Success_Rows(df_)
                  #print(leadway_success_concat_text)
                  res_text_leadway = parse_Leadway_Success_Row(leadway_success_concat_text)
                  #print(res_text_leadway)
                  for o in range(len(list_column_none_leadway_success)):
                    list_column_none_leadway_success[o].append(None)
                  
                  #print('column none leadway success:')
                  #print(list_column_none_leadway_success)

                  try:
                    type_request_col.append(type_of_request.group(0))
                  except AttributeError:
                    type_request_col.append(None)
                 


    elif re.search('error', str_text):
        log_level = re.search('error', str_text)
          
        try:
          log_level_col.append(log_level.group(0))
        except AttributeError:
          log_level_col.append(None)

        if re.search('LOAN ERROR', str_text):
            type_of_request = re.search('LOAN ERROR', str_text)
            loan_error = parse_Error_Row(str_text) 
            try:
              error_code_col.append(loan_error.get('code'))
              print(error_code_col)
            except AttributeError:
              error_code_col.append(None)
            try:
              print(loan_error.get('errno'))
              error_number_col.append(loan_error.get('errno'))
              print('error number :')
              print(error_number_col)
            except AttributeError:
              error_number_col.append(None)
            try:
              error_sql_message_col.append(loan_error.get('sqlMessage'))
            except AttributeError:
              error_sql_message_col.append(None)
            try:
              error_sql_state_col.append(loan_error.get('sqlState'))
            except AttributeError:
              error_sql_state_col.append(None)
            try:
              error_index_col.append(loan_error.get('index'))
            except AttributeError:
              error_index_col.append(None)
            try:
              error_sql_col.append(loan_error.get('sql'))
            except AttributeError:
              error_sql_col.append(None)       

            try:
                type_request_col.append(type_of_request.group(0))
            except AttributeError:
                type_request_col.append(None)

            for p in range(len(list_column_none_level_log_error)):
              list_column_none_level_log_error[p].append(None)

            

        elif re.search('OKRA PAYLOAD', str_text): # Nothing
          type_of_request = re.search('OKRA PAYLOAD', str_text)
        elif re.search('OKRA SUCCESS', str_text):   # Nothing
          type_of_request = re.search('OKRA SUCCESS', str_text)
        elif re.search('VTPASS SUCCESS', str_text):   # Nothing
          type_of_request = re.search('VTPASS SUCCESS', str_text)  


  
  
  #for x in range(len(list_all_colum)):
   # print(len(list_all_colum[x]))
  

  df_['Type_Request'] = type_request_col
  df_['Phone_Number'] = phone_Col
  df_['Date'] = date_col
  df_['EndPoint'] = endpoint_Col
  df_['Log_Level'] = log_level_col
  df_['Email'] = email_col
  df_['Message SMS Payload'] = message_sms_payload_col
  df_['Total Sent'] = totalsent_col
  df_['Cost'] = cost_col
  df_['Status'] = status_col
  df_['Account Number'] = account_number_col
  df_['Account Name'] = account_name_col
  df_['BVN'] = bvn_col
  df_['Request Successful'] = requestSuccessful_col
  df_['Response Message'] = responseMessage_col
  df_['Response Code'] = responseCode_col
  df_['Error Code'] = error_code_col
  df_['Error Number'] = error_number_col
  df_['Error Sql Message'] = error_sql_message_col
  df_['Error Sql State'] = error_sql_state_col
  df_['Error Index'] = error_index_col
  df_['Error Sql'] = error_sql_col


  return df_




In [ ]:
df_error = df_raw[df_raw['text'].str.contains('LOAN ERROR')]
resutat_error = parse_row_error(df_error)
resutat.head()

In [ ]:
parserow(df_raw)

In [60]:
df_raw = df_raw.drop(2355)

In [61]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4952 entries, 0 to 4952
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   type     4952 non-null   object
 1   subtype  4952 non-null   object
 2   text     4952 non-null   object
 3   ts       4952 non-null   object
 4   bot_id   4952 non-null   object
dtypes: object(5)
memory usage: 232.1+ KB


### Handle case type is API REQUEST

In [67]:
import re
import regex
import json

def parse_row_api_request(df_api_request):
  log_level_col = []
  api_request_col = []
  type_request_col = []
  phone_Col = []
  date_col = []
  endpoint_Col = []
  email_col = []
  message_sms_payload_col = []
  totalsent_col = []
  cost_col = []
  status_col = []
  account_number_col = []
  account_name_col = []
  bvn_col = []
  requestSuccessful_col = []
  responseMessage_col = []
  responseCode_col = []

  list_column_none_api_request = []
  list_column_none_api_request.append(message_sms_payload_col)
  list_column_none_api_request.append(totalsent_col)
  list_column_none_api_request.append(cost_col)
  list_column_none_api_request.append(status_col)
  list_column_none_api_request.append(account_number_col)
  list_column_none_api_request.append(bvn_col)
  list_column_none_api_request.append(requestSuccessful_col)
  list_column_none_api_request.append(responseMessage_col)
  list_column_none_api_request.append(responseCode_col)
  list_column_none_api_request.append(account_name_col)
  
  list_column_none_client_mobile = []
  list_column_none_client_mobile.append(message_sms_payload_col)
  list_column_none_client_mobile.append(totalsent_col)
  list_column_none_client_mobile.append(cost_col)
  list_column_none_client_mobile.append(status_col)
  list_column_none_client_mobile.append(account_number_col)
  list_column_none_client_mobile.append(bvn_col)
  list_column_none_client_mobile.append(requestSuccessful_col)
  list_column_none_client_mobile.append(responseMessage_col)
  list_column_none_client_mobile.append(responseCode_col)
  list_column_none_client_mobile.append(account_name_col)
  list_column_none_client_mobile.append(endpoint_Col)

  list_column_none_sms_payload = []
  list_column_none_sms_payload.append(totalsent_col)
  list_column_none_sms_payload.append(cost_col)
  list_column_none_sms_payload.append(status_col)
  list_column_none_sms_payload.append(account_number_col)
  list_column_none_sms_payload.append(bvn_col)
  list_column_none_sms_payload.append(requestSuccessful_col)
  list_column_none_sms_payload.append(responseMessage_col)
  list_column_none_sms_payload.append(responseCode_col)
  list_column_none_sms_payload.append(account_name_col)
  list_column_none_sms_payload.append(email_col)
  list_column_none_sms_payload.append(endpoint_Col)
  list_column_none_sms_payload.append(date_col)

  list_column_none_sms_success = []
  list_column_none_sms_success.append(message_sms_payload_col)
  list_column_none_sms_success.append(account_number_col)
  list_column_none_sms_success.append(bvn_col)
  list_column_none_sms_success.append(requestSuccessful_col)
  list_column_none_sms_success.append(responseMessage_col)
  list_column_none_sms_success.append(responseCode_col)
  list_column_none_sms_success.append(account_name_col)
  list_column_none_sms_success.append(email_col)
  list_column_none_sms_success.append(phone_Col)
  list_column_none_sms_success.append(endpoint_Col)
  list_column_none_sms_success.append(date_col)


  list_column_none_wallet_success = []
  list_column_none_wallet_success.append(totalsent_col)
  list_column_none_wallet_success.append(message_sms_payload_col)
  list_column_none_wallet_success.append(cost_col)
  list_column_none_wallet_success.append(status_col)
  list_column_none_wallet_success.append(email_col)
  list_column_none_wallet_success.append(phone_Col)
  list_column_none_wallet_success.append(endpoint_Col)
  list_column_none_wallet_success.append(date_col)


  list_column_none_leadway_success = []
  list_column_none_leadway_success.append(message_sms_payload_col)
  list_column_none_leadway_success.append(totalsent_col)
  list_column_none_leadway_success.append(cost_col)
  list_column_none_leadway_success.append(status_col)
  list_column_none_leadway_success.append(account_number_col)
  list_column_none_leadway_success.append(bvn_col)
  list_column_none_leadway_success.append(requestSuccessful_col)
  list_column_none_leadway_success.append(responseMessage_col)
  list_column_none_leadway_success.append(responseCode_col)
  list_column_none_leadway_success.append(account_name_col)
  list_column_none_leadway_success.append(email_col)
  list_column_none_leadway_success.append(phone_Col)
  list_column_none_leadway_success.append(endpoint_Col)
  list_column_none_leadway_success.append(date_col)

  
  
  list_all_colum = []
  list_all_colum.append(type_request_col)
  list_all_colum.append(phone_Col)
  list_all_colum.append(date_col)
  list_all_colum.append(endpoint_Col)
  list_all_colum.append(log_level_col)
  list_all_colum.append(email_col)
  list_all_colum.append(message_sms_payload_col)
  list_all_colum.append(totalsent_col)
  list_all_colum.append(cost_col)
  list_all_colum.append(status_col)
  list_all_colum.append(account_number_col)
  list_all_colum.append(account_name_col)
  list_all_colum.append(bvn_col)
  list_all_colum.append(requestSuccessful_col)
  list_all_colum.append(responseMessage_col)
  list_all_colum.append(responseCode_col)

  for index, row in df_api_request.iterrows():
    str_text = row['text']
    
    if not str_text.startswith('['):
      for i in range(len(list_all_colum)):
          list_all_colum[i].append(None)

    # check if the row contains "info" string
    if re.search('info', str_text):
        log_level = re.search('info', str_text)
        try:
          log_level_col.append(log_level.group(0))
        except AttributeError:
          log_level_col.append(None)        
        # check if the row contains an email address 
        # pour tous les types request créer un dictionnaire dans lequel mapper
        # key = type de request et value = les regex définis
        # pour chaque condition IF créer une liste de colonnes auxquelles affecter None
        if 'mailto' in str_text:
            if re.search('API REQUEST', str_text):
                type_of_request = re.search('API REQUEST', str_text)                
                phone_or_email_api_req = re.search(type_request_dictionnary['API REQUEST'][0], str_text)                              
                endpoint = re.search(type_request_dictionnary['API REQUEST'][1], str_text)
                pattern = type_request_dictionnary['API REQUEST'][2]
                datepattern = re.compile("(?:%s)"%(pattern))
                datematcher = datepattern.search(str_text)  # extract date

                for i in range(len(list_column_none_api_request)):
                  list_column_none_api_request[i].append(None)
                
                #print('column none api request:')
                #print(list_column_none_api_request)
                               
                try:
                  type_request_col.append(type_of_request.group(0)) # add type request inside type request column
                except AttributeError:
                  type_request_col.append(None)               
                try:
                  email_col.append(phone_or_email_api_req.group(0)) # add email inside email column
                  phone_Col.append(None)  # in this case there is no phone number
                except AttributeError:
                  email_col.append(None)
                try:
                  endpoint_Col.append(endpoint.group(0)) # add endpoint inside endpoint column
                except AttributeError:
                  endpoint_Col.append(None)
                try:
                  date_col.append(convertToTimestamp(datematcher.group(0))) # convert date to timestamp and add it inside date column
                except AttributeError:
                  date_col.append(None)
              

        elif 'mailto' not in str_text:
            if re.search('API REQUEST', str_text):
                type_of_request = re.search('API REQUEST', str_text)                            
                # extract a phone number for API REQUEST
                phone_or_email_api_req = re.search(type_request_dictionnary['API REQUEST'][3], str_text)                              
                endpoint = re.search(type_request_dictionnary['API REQUEST'][1], str_text)
                pattern = type_request_dictionnary['API REQUEST'][2]
                datepattern = re.compile("(?:%s)"%(pattern))
                datematcher = datepattern.search(str_text)  # extract date

                for i in range(len(list_column_none_api_request)):
                  list_column_none_api_request[i].append(None)
                
                #print('column none api request no mail:')
                #print(list_column_none_api_request)
                try:
                  phone_Col.append(phone_or_email_api_req.group(0)) # add phone number inside phone number column
                  email_col.append(None) # in this case there is no email address
                except AttributeError:
                  phone_Col.append(None)
                try:
                  type_request_col.append(type_of_request.group(0))
                except AttributeError:
                  type_request_col.append(None)
                try:
                  endpoint_Col.append(endpoint.group(0)) # add endpoint inside endpoint column
                except AttributeError:
                  endpoint_Col.append(None)
                try:
                  date_col.append(convertToTimestamp(datematcher.group(0))) # convert date to timestamp and add it inside date column
                except AttributeError:
                  date_col.append(None)

  df_api_request['Type_Request'] = type_request_col
  df_api_request['Phone_Number'] = phone_Col
  df_api_request['Date'] = date_col
  df_api_request['EndPoint'] = endpoint_Col
  df_api_request['Log_Level'] = log_level_col
  df_api_request['Email'] = email_col
  df_api_request['Message SMS Payload'] = message_sms_payload_col
  df_api_request['Total Sent'] = totalsent_col
  df_api_request['Cost'] = cost_col
  df_api_request['Status'] = status_col
  df_api_request['Account Number'] = account_number_col
  df_api_request['Account Name'] = account_name_col
  df_api_request['BVN'] = bvn_col
  df_api_request['Request Successful'] = requestSuccessful_col
  df_api_request['Response Message'] = responseMessage_col
  df_api_request['Response Code'] = responseCode_col
 


  return df_api_request




In [ ]:
df_api_request_ = df_raw[df_raw['text'].str.contains('API REQUEST')]
resutat_df_api = parse_row_error(df_api_request_)
resutat_df_api.head(50)
resutat_df_api.info()

### Handle Client Mobile Login

In [76]:
import re
import regex
import json



def parse_row_client_mobile_login(df_client_mob):
  log_level_col = []
  api_request_col = []
  type_request_col = []
  phone_Col = []
  date_col = []
  endpoint_Col = []
  email_col = []
  message_sms_payload_col = []
  totalsent_col = []
  cost_col = []
  status_col = []
  account_number_col = []
  account_name_col = []
  bvn_col = []
  requestSuccessful_col = []
  responseMessage_col = []
  responseCode_col = []


  list_column_none_api_request = []
  list_column_none_api_request.append(message_sms_payload_col)
  list_column_none_api_request.append(totalsent_col)
  list_column_none_api_request.append(cost_col)
  list_column_none_api_request.append(status_col)
  list_column_none_api_request.append(account_number_col)
  list_column_none_api_request.append(bvn_col)
  list_column_none_api_request.append(requestSuccessful_col)
  list_column_none_api_request.append(responseMessage_col)
  list_column_none_api_request.append(responseCode_col)
  list_column_none_api_request.append(account_name_col)
  
  list_column_none_client_mobile = []
  list_column_none_client_mobile.append(message_sms_payload_col)
  list_column_none_client_mobile.append(totalsent_col)
  list_column_none_client_mobile.append(cost_col)
  list_column_none_client_mobile.append(status_col)
  list_column_none_client_mobile.append(account_number_col)
  list_column_none_client_mobile.append(bvn_col)
  list_column_none_client_mobile.append(requestSuccessful_col)
  list_column_none_client_mobile.append(responseMessage_col)
  list_column_none_client_mobile.append(responseCode_col)
  list_column_none_client_mobile.append(account_name_col)
  list_column_none_client_mobile.append(endpoint_Col)

  list_column_none_sms_payload = []
  list_column_none_sms_payload.append(totalsent_col)
  list_column_none_sms_payload.append(cost_col)
  list_column_none_sms_payload.append(status_col)
  list_column_none_sms_payload.append(account_number_col)
  list_column_none_sms_payload.append(bvn_col)
  list_column_none_sms_payload.append(requestSuccessful_col)
  list_column_none_sms_payload.append(responseMessage_col)
  list_column_none_sms_payload.append(responseCode_col)
  list_column_none_sms_payload.append(account_name_col)
  list_column_none_sms_payload.append(email_col)
  list_column_none_sms_payload.append(endpoint_Col)
  list_column_none_sms_payload.append(date_col)

  list_column_none_sms_success = []
  list_column_none_sms_success.append(message_sms_payload_col)
  list_column_none_sms_success.append(account_number_col)
  list_column_none_sms_success.append(bvn_col)
  list_column_none_sms_success.append(requestSuccessful_col)
  list_column_none_sms_success.append(responseMessage_col)
  list_column_none_sms_success.append(responseCode_col)
  list_column_none_sms_success.append(account_name_col)
  list_column_none_sms_success.append(email_col)
  list_column_none_sms_success.append(phone_Col)
  list_column_none_sms_success.append(endpoint_Col)
  list_column_none_sms_success.append(date_col)


  list_column_none_wallet_success = []
  list_column_none_wallet_success.append(totalsent_col)
  list_column_none_wallet_success.append(message_sms_payload_col)
  list_column_none_wallet_success.append(cost_col)
  list_column_none_wallet_success.append(status_col)
  list_column_none_wallet_success.append(email_col)
  list_column_none_wallet_success.append(phone_Col)
  list_column_none_wallet_success.append(endpoint_Col)
  list_column_none_wallet_success.append(date_col)


  list_column_none_leadway_success = []
  list_column_none_leadway_success.append(message_sms_payload_col)
  list_column_none_leadway_success.append(totalsent_col)
  list_column_none_leadway_success.append(cost_col)
  list_column_none_leadway_success.append(status_col)
  list_column_none_leadway_success.append(account_number_col)
  list_column_none_leadway_success.append(bvn_col)
  list_column_none_leadway_success.append(requestSuccessful_col)
  list_column_none_leadway_success.append(responseMessage_col)
  list_column_none_leadway_success.append(responseCode_col)
  list_column_none_leadway_success.append(account_name_col)
  list_column_none_leadway_success.append(email_col)
  list_column_none_leadway_success.append(phone_Col)
  list_column_none_leadway_success.append(endpoint_Col)
  list_column_none_leadway_success.append(date_col)

  
  
  list_all_colum = []
  list_all_colum.append(type_request_col)
  list_all_colum.append(phone_Col)
  list_all_colum.append(date_col)
  list_all_colum.append(endpoint_Col)
  list_all_colum.append(log_level_col)
  list_all_colum.append(email_col)
  list_all_colum.append(message_sms_payload_col)
  list_all_colum.append(totalsent_col)
  list_all_colum.append(cost_col)
  list_all_colum.append(status_col)
  list_all_colum.append(account_number_col)
  list_all_colum.append(account_name_col)
  list_all_colum.append(bvn_col)
  list_all_colum.append(requestSuccessful_col)
  list_all_colum.append(responseMessage_col)
  list_all_colum.append(responseCode_col)

  for index, row in df_client_mob.iterrows():
    str_text = row['text']
    
    if not str_text.startswith('['):
      for i in range(len(list_all_colum)):
          list_all_colum[i].append(None)

    # check if the row contains "info" string
    if re.search('info', str_text):
        log_level = re.search('info', str_text)
        try:
          log_level_col.append(log_level.group(0))
        except AttributeError:
          log_level_col.append(None)        
        # check if the row contains an email address 
        # pour tous les types request créer un dictionnaire dans lequel mapper
        # key = type de request et value = les regex définis
        # pour chaque condition IF créer une liste de colonnes auxquelles affecter None
        if 'mailto' in str_text:
            if re.search('CLIENT MOBILE LOGIN', str_text):   # CLIENT MOBILE LOGIN with email address
                  type_of_request = re.search('CLIENT MOBILE LOGIN', str_text)
                  # extract address email for CLIENT MOBILE LOGIN
                  phone_or_email_client_mobile = re.search(type_request_dictionnary['CLIENT MOBILE LOGIN'][0], str_text)                               
                  pattern = type_request_dictionnary['CLIENT MOBILE LOGIN'][1]
                  datepattern = re.compile("(?:%s)"%(pattern))
                  datematcher = datepattern.search(str_text)  # extract date for CLIENT MOBILE LOGIN type request
                  
                  for j in range(len(list_column_none_client_mobile)):
                    list_column_none_client_mobile[j].append(None)

                  #print('column none client mobile:')
                  #print(list_column_none_client_mobile)
                  try:
                    type_request_col.append(type_of_request.group(0)) # add type request inside type request column
                  except AttributeError:
                    type_request_col.append(None) 
                  try:
                    email_col.append(phone_or_email_client_mobile.group(0)) # add email inside email column
                    phone_Col.append(None)  # in this case there is no phone number
                  except AttributeError:
                    email_col.append(None)
                  try:
                    date_col.append(convertToTimestamp(datematcher.group(0))) # convert date to timestamp and add it inside date column
                  except AttributeError:
                    date_col.append(None)

        elif 'mailto' not in str_text:
            if re.search('CLIENT MOBILE LOGIN', str_text): # when type request is CLIENT MOBILE LOGIN, there is no EndPoint
                type_of_request = re.search('CLIENT MOBILE LOGIN', str_text)
                # extract a phone number for CLIENT MOBILE LOGIN
                phone_or_email_client_mobile = re.search(type_request_dictionnary['CLIENT MOBILE LOGIN'][2], str_text)                  
                pattern = type_request_dictionnary['CLIENT MOBILE LOGIN'][1]
                datepattern = re.compile("(?:%s)"%(pattern))
                datematcher = datepattern.search(str_text)  # extract date

                for j in range(len(list_column_none_client_mobile)):
                    list_column_none_client_mobile[j].append(None)

                #print('column none client mobile no mail:')
                #print(list_column_none_client_mobile)
                try:
                  phone_Col.append(phone_or_email_client_mobile.group(0))
                  email_col.append(None)
                except AttributeError:
                  phone_Col.append(None)
                try:
                  type_request_col.append(type_of_request.group(0))
                except AttributeError:
                  type_request_col.append(None)
                try:
                  date_col.append(convertToTimestamp(datematcher.group(0))) # convert date to timestamp and add it inside date column
                except AttributeError:
                  date_col.append(None)
  
  #for x in range(len(list_all_colum)):
    #print(len(list_all_colum[x]))
  

  df_client_mob['Type_Request'] = type_request_col
  df_client_mob['Phone_Number'] = phone_Col
  df_client_mob['Date'] = date_col
  df_client_mob['EndPoint'] = endpoint_Col
  df_client_mob['Log_Level'] = log_level_col
  df_client_mob['Email'] = email_col
  df_client_mob['Message SMS Payload'] = message_sms_payload_col
  df_client_mob['Total Sent'] = totalsent_col
  df_client_mob['Cost'] = cost_col
  df_client_mob['Status'] = status_col
  df_client_mob['Account Number'] = account_number_col
  df_client_mob['Account Name'] = account_name_col
  df_client_mob['BVN'] = bvn_col
  df_client_mob['Request Successful'] = requestSuccessful_col
  df_client_mob['Response Message'] = responseMessage_col
  df_client_mob['Response Code'] = responseCode_col
 


  return df_client_mob


In [ ]:
df_client_mobile_login = df_raw[df_raw['text'].str.contains('CLIENT MOBILE LOGIN')]
resutat_df_client_mobile_login = parse_row_client_mobile_login(df_client_mobile_login)
resutat_df_client_mobile_login.head()
#resutat_df_client_mobile_login.info()



### Handle SMS PAYLOAD Type request 

In [79]:
import re
import regex
import json

# parse row of text column

def parse_row_sms_payload_function(df_sms_payload):
  log_level_col = []
  api_request_col = []
  type_request_col = []
  phone_Col = []
  date_col = []
  endpoint_Col = []
  email_col = []
  message_sms_payload_col = []
  totalsent_col = []
  cost_col = []
  status_col = []
  account_number_col = []
  account_name_col = []
  bvn_col = []
  requestSuccessful_col = []
  responseMessage_col = []
  responseCode_col = []


  list_column_none_api_request = []
  list_column_none_api_request.append(message_sms_payload_col)
  list_column_none_api_request.append(totalsent_col)
  list_column_none_api_request.append(cost_col)
  list_column_none_api_request.append(status_col)
  list_column_none_api_request.append(account_number_col)
  list_column_none_api_request.append(bvn_col)
  list_column_none_api_request.append(requestSuccessful_col)
  list_column_none_api_request.append(responseMessage_col)
  list_column_none_api_request.append(responseCode_col)
  list_column_none_api_request.append(account_name_col)
  
  list_column_none_client_mobile = []
  list_column_none_client_mobile.append(message_sms_payload_col)
  list_column_none_client_mobile.append(totalsent_col)
  list_column_none_client_mobile.append(cost_col)
  list_column_none_client_mobile.append(status_col)
  list_column_none_client_mobile.append(account_number_col)
  list_column_none_client_mobile.append(bvn_col)
  list_column_none_client_mobile.append(requestSuccessful_col)
  list_column_none_client_mobile.append(responseMessage_col)
  list_column_none_client_mobile.append(responseCode_col)
  list_column_none_client_mobile.append(account_name_col)
  list_column_none_client_mobile.append(endpoint_Col)

  list_column_none_sms_payload = []
  list_column_none_sms_payload.append(totalsent_col)
  list_column_none_sms_payload.append(cost_col)
  list_column_none_sms_payload.append(status_col)
  list_column_none_sms_payload.append(account_number_col)
  list_column_none_sms_payload.append(bvn_col)
  list_column_none_sms_payload.append(requestSuccessful_col)
  list_column_none_sms_payload.append(responseMessage_col)
  list_column_none_sms_payload.append(responseCode_col)
  list_column_none_sms_payload.append(account_name_col)
  list_column_none_sms_payload.append(email_col)
  list_column_none_sms_payload.append(endpoint_Col)
  list_column_none_sms_payload.append(date_col)

  list_column_none_sms_success = []
  list_column_none_sms_success.append(message_sms_payload_col)
  list_column_none_sms_success.append(account_number_col)
  list_column_none_sms_success.append(bvn_col)
  list_column_none_sms_success.append(requestSuccessful_col)
  list_column_none_sms_success.append(responseMessage_col)
  list_column_none_sms_success.append(responseCode_col)
  list_column_none_sms_success.append(account_name_col)
  list_column_none_sms_success.append(email_col)
  list_column_none_sms_success.append(phone_Col)
  list_column_none_sms_success.append(endpoint_Col)
  list_column_none_sms_success.append(date_col)


  list_column_none_wallet_success = []
  list_column_none_wallet_success.append(totalsent_col)
  list_column_none_wallet_success.append(message_sms_payload_col)
  list_column_none_wallet_success.append(cost_col)
  list_column_none_wallet_success.append(status_col)
  list_column_none_wallet_success.append(email_col)
  list_column_none_wallet_success.append(phone_Col)
  list_column_none_wallet_success.append(endpoint_Col)
  list_column_none_wallet_success.append(date_col)


  list_column_none_leadway_success = []
  list_column_none_leadway_success.append(message_sms_payload_col)
  list_column_none_leadway_success.append(totalsent_col)
  list_column_none_leadway_success.append(cost_col)
  list_column_none_leadway_success.append(status_col)
  list_column_none_leadway_success.append(account_number_col)
  list_column_none_leadway_success.append(bvn_col)
  list_column_none_leadway_success.append(requestSuccessful_col)
  list_column_none_leadway_success.append(responseMessage_col)
  list_column_none_leadway_success.append(responseCode_col)
  list_column_none_leadway_success.append(account_name_col)
  list_column_none_leadway_success.append(email_col)
  list_column_none_leadway_success.append(phone_Col)
  list_column_none_leadway_success.append(endpoint_Col)
  list_column_none_leadway_success.append(date_col)

  
  
  list_all_colum = []
  list_all_colum.append(type_request_col)
  list_all_colum.append(phone_Col)
  list_all_colum.append(date_col)
  list_all_colum.append(endpoint_Col)
  list_all_colum.append(log_level_col)
  list_all_colum.append(email_col)
  list_all_colum.append(message_sms_payload_col)
  list_all_colum.append(totalsent_col)
  list_all_colum.append(cost_col)
  list_all_colum.append(status_col)
  list_all_colum.append(account_number_col)
  list_all_colum.append(account_name_col)
  list_all_colum.append(bvn_col)
  list_all_colum.append(requestSuccessful_col)
  list_all_colum.append(responseMessage_col)
  list_all_colum.append(responseCode_col)

  for index, row in df_sms_payload.iterrows():
    str_text = row['text']
    
    if not str_text.startswith('['):
      for i in range(len(list_all_colum)):
          list_all_colum[i].append(None)

    # check if the row contains "info" string
    if re.search('info', str_text):
        log_level = re.search('info', str_text)
        try:
          log_level_col.append(log_level.group(0))
        except AttributeError:
          log_level_col.append(None)             
        if 'mailto' not in str_text:
            if re.search('SMS PAYLOAD', str_text):
                type_of_request = re.search('SMS PAYLOAD', str_text)            
                sms_payload = parse_wallet_sms_payload_success(str_text)               
                for l in range(len(list_column_none_sms_payload)):
                    list_column_none_sms_payload[l].append(None)             
                try:
                  type_request_col.append(type_of_request.group(0))
                except AttributeError:
                  type_request_col.append(None)
                try:
                  phone_Col.append(sms_payload.get('phone'))
                except AttributeError:
                  phone_Col.append(None)
                try:
                  message_sms_payload_col.append(sms_payload.get('message'))
                except AttributeError:
                  message_sms_payload_col.append(None)
                     
        elif re.search('OKRA PAYLOAD', str_text): # Nothing
          type_of_request = re.search('OKRA PAYLOAD', str_text)
        elif re.search('OKRA SUCCESS', str_text):   # Nothing
          type_of_request = re.search('OKRA SUCCESS', str_text)
        elif re.search('VTPASS SUCCESS', str_text):   # Nothing
          type_of_request = re.search('VTPASS SUCCESS', str_text)  

  for x in range(len(list_all_colum)):
    print(len(list_all_colum[x]))
  

  df_sms_payload['Type_Request'] = type_request_col
  df_sms_payload['Phone_Number'] = phone_Col
  df_sms_payload['Date'] = date_col
  df_sms_payload['EndPoint'] = endpoint_Col
  df_sms_payload['Log_Level'] = log_level_col
  df_sms_payload['Email'] = email_col
  df_sms_payload['Message SMS Payload'] = message_sms_payload_col
  df_sms_payload['Total Sent'] = totalsent_col
  df_sms_payload['Cost'] = cost_col
  df_sms_payload['Status'] = status_col
  df_sms_payload['Account Number'] = account_number_col
  df_sms_payload['Account Name'] = account_name_col
  df_sms_payload['BVN'] = bvn_col
  df_sms_payload['Request Successful'] = requestSuccessful_col
  df_sms_payload['Response Message'] = responseMessage_col
  df_sms_payload['Response Code'] = responseCode_col
 
  return df_sms_payload


In [ ]:
df_sms_payload_ = df_raw[df_raw['text'].str.contains('SMS PAYLOAD')]
resutat_df_sms_payload = parse_row_sms_payload_function(df_sms_payload_)
resutat_df_sms_payload.head()
#resutat_df_sms_payload.info()

### Handel SMS Success Type request

In [81]:
import re
import regex
import json

# parse row of text column

def parse_row_sms_success_function(df_sms_success):
  log_level_col = []
  api_request_col = []
  type_request_col = []
  phone_Col = []
  date_col = []
  endpoint_Col = []
  email_col = []
  message_sms_payload_col = []
  totalsent_col = []
  cost_col = []
  status_col = []
  account_number_col = []
  account_name_col = []
  bvn_col = []
  requestSuccessful_col = []
  responseMessage_col = []
  responseCode_col = []


  list_column_none_api_request = []
  list_column_none_api_request.append(message_sms_payload_col)
  list_column_none_api_request.append(totalsent_col)
  list_column_none_api_request.append(cost_col)
  list_column_none_api_request.append(status_col)
  list_column_none_api_request.append(account_number_col)
  list_column_none_api_request.append(bvn_col)
  list_column_none_api_request.append(requestSuccessful_col)
  list_column_none_api_request.append(responseMessage_col)
  list_column_none_api_request.append(responseCode_col)
  list_column_none_api_request.append(account_name_col)
  
  list_column_none_client_mobile = []
  list_column_none_client_mobile.append(message_sms_payload_col)
  list_column_none_client_mobile.append(totalsent_col)
  list_column_none_client_mobile.append(cost_col)
  list_column_none_client_mobile.append(status_col)
  list_column_none_client_mobile.append(account_number_col)
  list_column_none_client_mobile.append(bvn_col)
  list_column_none_client_mobile.append(requestSuccessful_col)
  list_column_none_client_mobile.append(responseMessage_col)
  list_column_none_client_mobile.append(responseCode_col)
  list_column_none_client_mobile.append(account_name_col)
  list_column_none_client_mobile.append(endpoint_Col)

  list_column_none_sms_payload = []
  list_column_none_sms_payload.append(totalsent_col)
  list_column_none_sms_payload.append(cost_col)
  list_column_none_sms_payload.append(status_col)
  list_column_none_sms_payload.append(account_number_col)
  list_column_none_sms_payload.append(bvn_col)
  list_column_none_sms_payload.append(requestSuccessful_col)
  list_column_none_sms_payload.append(responseMessage_col)
  list_column_none_sms_payload.append(responseCode_col)
  list_column_none_sms_payload.append(account_name_col)
  list_column_none_sms_payload.append(email_col)
  list_column_none_sms_payload.append(endpoint_Col)
  list_column_none_sms_payload.append(date_col)

  list_column_none_sms_success = []
  list_column_none_sms_success.append(message_sms_payload_col)
  list_column_none_sms_success.append(account_number_col)
  list_column_none_sms_success.append(bvn_col)
  list_column_none_sms_success.append(requestSuccessful_col)
  list_column_none_sms_success.append(responseMessage_col)
  list_column_none_sms_success.append(responseCode_col)
  list_column_none_sms_success.append(account_name_col)
  list_column_none_sms_success.append(email_col)
  list_column_none_sms_success.append(phone_Col)
  list_column_none_sms_success.append(endpoint_Col)
  list_column_none_sms_success.append(date_col)


  list_column_none_wallet_success = []
  list_column_none_wallet_success.append(totalsent_col)
  list_column_none_wallet_success.append(message_sms_payload_col)
  list_column_none_wallet_success.append(cost_col)
  list_column_none_wallet_success.append(status_col)
  list_column_none_wallet_success.append(email_col)
  list_column_none_wallet_success.append(phone_Col)
  list_column_none_wallet_success.append(endpoint_Col)
  list_column_none_wallet_success.append(date_col)


  list_column_none_leadway_success = []
  list_column_none_leadway_success.append(message_sms_payload_col)
  list_column_none_leadway_success.append(totalsent_col)
  list_column_none_leadway_success.append(cost_col)
  list_column_none_leadway_success.append(status_col)
  list_column_none_leadway_success.append(account_number_col)
  list_column_none_leadway_success.append(bvn_col)
  list_column_none_leadway_success.append(requestSuccessful_col)
  list_column_none_leadway_success.append(responseMessage_col)
  list_column_none_leadway_success.append(responseCode_col)
  list_column_none_leadway_success.append(account_name_col)
  list_column_none_leadway_success.append(email_col)
  list_column_none_leadway_success.append(phone_Col)
  list_column_none_leadway_success.append(endpoint_Col)
  list_column_none_leadway_success.append(date_col)

  
  
  list_all_colum = []
  list_all_colum.append(type_request_col)
  list_all_colum.append(phone_Col)
  list_all_colum.append(date_col)
  list_all_colum.append(endpoint_Col)
  list_all_colum.append(log_level_col)
  list_all_colum.append(email_col)
  list_all_colum.append(message_sms_payload_col)
  list_all_colum.append(totalsent_col)
  list_all_colum.append(cost_col)
  list_all_colum.append(status_col)
  list_all_colum.append(account_number_col)
  list_all_colum.append(account_name_col)
  list_all_colum.append(bvn_col)
  list_all_colum.append(requestSuccessful_col)
  list_all_colum.append(responseMessage_col)
  list_all_colum.append(responseCode_col)

  for index, row in df_sms_success.iterrows():
    str_text = row['text']
    
    if not str_text.startswith('['):
      for i in range(len(list_all_colum)):
          list_all_colum[i].append(None)

    # check if the row contains "info" string
    if re.search('info', str_text):
        log_level = re.search('info', str_text)
        try:
          log_level_col.append(log_level.group(0))
        except AttributeError:
          log_level_col.append(None)             
        if 'mailto' not in str_text:
            if re.search('SMS SUCCESS', str_text): 
                type_of_request = re.search('SMS SUCCESS', str_text)
                sms_success = parse_wallet_sms_payload_success(str_text)                
                for m in range(len(list_column_none_sms_success)):
                    list_column_none_sms_success[m].append(None) 
                
                #print('column none sms success:')
                #print(list_column_none_sms_success)             
                try:
                  type_request_col.append(type_of_request.group(0))
                except AttributeError:
                  type_request_col.append(None)
                try:                 
                  totalsent_col.append(sms_success.get('response').get('totalsent '))
                except AttributeError:
                  totalsent_col.append(None)
                try:                 
                  cost_col.append(sms_success.get('response').get('cost '))
                except AttributeError:
                  cost_col.append(None)
                try:                 
                  status_col.append(sms_success.get('response').get('status '))
                except AttributeError:
                  status_col.append(None)
                     
        elif re.search('OKRA PAYLOAD', str_text): # Nothing
          type_of_request = re.search('OKRA PAYLOAD', str_text)
        elif re.search('OKRA SUCCESS', str_text):   # Nothing
          type_of_request = re.search('OKRA SUCCESS', str_text)
        elif re.search('VTPASS SUCCESS', str_text):   # Nothing
          type_of_request = re.search('VTPASS SUCCESS', str_text)  

  for x in range(len(list_all_colum)):
    print(len(list_all_colum[x]))
  

  df_sms_success['Type_Request'] = type_request_col
  df_sms_success['Phone_Number'] = phone_Col
  df_sms_success['Date'] = date_col
  df_sms_success['EndPoint'] = endpoint_Col
  df_sms_success['Log_Level'] = log_level_col
  df_sms_success['Email'] = email_col
  df_sms_success['Message SMS Payload'] = message_sms_payload_col
  df_sms_success['Total Sent'] = totalsent_col
  df_sms_success['Cost'] = cost_col
  df_sms_success['Status'] = status_col
  df_sms_success['Account Number'] = account_number_col
  df_sms_success['Account Name'] = account_name_col
  df_sms_success['BVN'] = bvn_col
  df_sms_success['Request Successful'] = requestSuccessful_col
  df_sms_success['Response Message'] = responseMessage_col
  df_sms_success['Response Code'] = responseCode_col
 
  return df_sms_success


In [ ]:
df_sms_success_ = df_raw[df_raw['text'].str.contains('SMS SUCCESS')]
resutat_df_sms_success = parse_row_sms_success_function(df_sms_success_)
resutat_df_sms_success.head()
#resutat_df_sms_success.info()

### Handle WALLET SUCCESS Type request

In [83]:
import re
import regex
import json

# parse row of text column

def parse_row_wallet_success_function(df_wallet_success):
  log_level_col = []
  api_request_col = []
  type_request_col = []
  phone_Col = []
  date_col = []
  endpoint_Col = []
  email_col = []
  message_sms_payload_col = []
  totalsent_col = []
  cost_col = []
  status_col = []
  account_number_col = []
  account_name_col = []
  bvn_col = []
  requestSuccessful_col = []
  responseMessage_col = []
  responseCode_col = []


  list_column_none_api_request = []
  list_column_none_api_request.append(message_sms_payload_col)
  list_column_none_api_request.append(totalsent_col)
  list_column_none_api_request.append(cost_col)
  list_column_none_api_request.append(status_col)
  list_column_none_api_request.append(account_number_col)
  list_column_none_api_request.append(bvn_col)
  list_column_none_api_request.append(requestSuccessful_col)
  list_column_none_api_request.append(responseMessage_col)
  list_column_none_api_request.append(responseCode_col)
  list_column_none_api_request.append(account_name_col)
  
  list_column_none_client_mobile = []
  list_column_none_client_mobile.append(message_sms_payload_col)
  list_column_none_client_mobile.append(totalsent_col)
  list_column_none_client_mobile.append(cost_col)
  list_column_none_client_mobile.append(status_col)
  list_column_none_client_mobile.append(account_number_col)
  list_column_none_client_mobile.append(bvn_col)
  list_column_none_client_mobile.append(requestSuccessful_col)
  list_column_none_client_mobile.append(responseMessage_col)
  list_column_none_client_mobile.append(responseCode_col)
  list_column_none_client_mobile.append(account_name_col)
  list_column_none_client_mobile.append(endpoint_Col)

  list_column_none_sms_payload = []
  list_column_none_sms_payload.append(totalsent_col)
  list_column_none_sms_payload.append(cost_col)
  list_column_none_sms_payload.append(status_col)
  list_column_none_sms_payload.append(account_number_col)
  list_column_none_sms_payload.append(bvn_col)
  list_column_none_sms_payload.append(requestSuccessful_col)
  list_column_none_sms_payload.append(responseMessage_col)
  list_column_none_sms_payload.append(responseCode_col)
  list_column_none_sms_payload.append(account_name_col)
  list_column_none_sms_payload.append(email_col)
  list_column_none_sms_payload.append(endpoint_Col)
  list_column_none_sms_payload.append(date_col)

  list_column_none_sms_success = []
  list_column_none_sms_success.append(message_sms_payload_col)
  list_column_none_sms_success.append(account_number_col)
  list_column_none_sms_success.append(bvn_col)
  list_column_none_sms_success.append(requestSuccessful_col)
  list_column_none_sms_success.append(responseMessage_col)
  list_column_none_sms_success.append(responseCode_col)
  list_column_none_sms_success.append(account_name_col)
  list_column_none_sms_success.append(email_col)
  list_column_none_sms_success.append(phone_Col)
  list_column_none_sms_success.append(endpoint_Col)
  list_column_none_sms_success.append(date_col)


  list_column_none_wallet_success = []
  list_column_none_wallet_success.append(totalsent_col)
  list_column_none_wallet_success.append(message_sms_payload_col)
  list_column_none_wallet_success.append(cost_col)
  list_column_none_wallet_success.append(status_col)
  list_column_none_wallet_success.append(email_col)
  list_column_none_wallet_success.append(phone_Col)
  list_column_none_wallet_success.append(endpoint_Col)
  list_column_none_wallet_success.append(date_col)


  list_column_none_leadway_success = []
  list_column_none_leadway_success.append(message_sms_payload_col)
  list_column_none_leadway_success.append(totalsent_col)
  list_column_none_leadway_success.append(cost_col)
  list_column_none_leadway_success.append(status_col)
  list_column_none_leadway_success.append(account_number_col)
  list_column_none_leadway_success.append(bvn_col)
  list_column_none_leadway_success.append(requestSuccessful_col)
  list_column_none_leadway_success.append(responseMessage_col)
  list_column_none_leadway_success.append(responseCode_col)
  list_column_none_leadway_success.append(account_name_col)
  list_column_none_leadway_success.append(email_col)
  list_column_none_leadway_success.append(phone_Col)
  list_column_none_leadway_success.append(endpoint_Col)
  list_column_none_leadway_success.append(date_col)

  
  
  list_all_colum = []
  list_all_colum.append(type_request_col)
  list_all_colum.append(phone_Col)
  list_all_colum.append(date_col)
  list_all_colum.append(endpoint_Col)
  list_all_colum.append(log_level_col)
  list_all_colum.append(email_col)
  list_all_colum.append(message_sms_payload_col)
  list_all_colum.append(totalsent_col)
  list_all_colum.append(cost_col)
  list_all_colum.append(status_col)
  list_all_colum.append(account_number_col)
  list_all_colum.append(account_name_col)
  list_all_colum.append(bvn_col)
  list_all_colum.append(requestSuccessful_col)
  list_all_colum.append(responseMessage_col)
  list_all_colum.append(responseCode_col)

  for index, row in df_wallet_success.iterrows():
    str_text = row['text']
    
    if not str_text.startswith('['):
      for i in range(len(list_all_colum)):
          list_all_colum[i].append(None)

    # check if the row contains "info" string
    if re.search('info', str_text):
        log_level = re.search('info', str_text)
        try:
          log_level_col.append(log_level.group(0))
        except AttributeError:
          log_level_col.append(None)             
        if 'mailto' not in str_text:
            if re.search('WALLET SUCCESS', str_text):
                  wallet_success = parse_wallet_sms_payload_success(str_text)
                  type_of_request = re.search('WALLET SUCCESS', str_text)

                  try:
                    type_request_col.append(type_of_request.group(0))
                  except AttributeError:
                    type_request_col.append(None)
                  try:
                    account_number_col.append(wallet_success.get('account_number'))
                  except AttributeError:
                    account_number_col.append(None)
                  try:
                    account_name_col.append(wallet_success.get('account_name'))
                  except AttributeError:
                    account_name_col.append(None)
                  try:
                    bvn_col.append(wallet_success.get('bvn'))
                  except AttributeError:
                    bvn_col.append(None)
                  try:
                    requestSuccessful_col.append(wallet_success.get('requestSuccessful'))
                  except AttributeError:
                    requestSuccessful_col.append(None)
                  try:
                    responseMessage_col.append((wallet_success.get('responseMessage')))
                  except AttributeError:
                    responseMessage_col.append(None)
                  try:
                    responseCode_col.append((wallet_success.get('responseCode')))
                  except AttributeError:
                    responseCode_col.append(None)
                  for n in range(len(list_column_none_wallet_success)):
                    list_column_none_wallet_success[n].append(None) 

                     
        elif re.search('OKRA PAYLOAD', str_text): # Nothing
          type_of_request = re.search('OKRA PAYLOAD', str_text)
        elif re.search('OKRA SUCCESS', str_text):   # Nothing
          type_of_request = re.search('OKRA SUCCESS', str_text)
        elif re.search('VTPASS SUCCESS', str_text):   # Nothing
          type_of_request = re.search('VTPASS SUCCESS', str_text)  

  #for x in range(len(list_all_colum)):
   # print(len(list_all_colum[x]))
  

  df_wallet_success['Type_Request'] = type_request_col
  df_wallet_success['Phone_Number'] = phone_Col
  df_wallet_success['Date'] = date_col
  df_wallet_success['EndPoint'] = endpoint_Col
  df_wallet_success['Log_Level'] = log_level_col
  df_wallet_success['Email'] = email_col
  df_wallet_success['Message SMS Payload'] = message_sms_payload_col
  df_wallet_success['Total Sent'] = totalsent_col
  df_wallet_success['Cost'] = cost_col
  df_wallet_success['Status'] = status_col
  df_wallet_success['Account Number'] = account_number_col
  df_wallet_success['Account Name'] = account_name_col
  df_wallet_success['BVN'] = bvn_col
  df_wallet_success['Request Successful'] = requestSuccessful_col
  df_wallet_success['Response Message'] = responseMessage_col
  df_wallet_success['Response Code'] = responseCode_col
 
  return df_wallet_success
             

In [ ]:
df_wallet_success_ = df_raw[df_raw['text'].str.contains('WALLET SUCCESS')]
resutat_df_wallet_success = parse_row_wallet_success_function(df_wallet_success_)
resutat_df_wallet_success.head()
#resutat_df_wallet_success.info()

### Handle OKRA WEBHOOK type request

In [85]:
import re
import regex
import json

def parse_row_okra_webhook_function(df_okra):
  log_level_col = []
  api_request_col = []
  type_request_col = []
  phone_Col = []
  date_col = []
  endpoint_Col = []
  email_col = []
  message_sms_payload_col = []
  totalsent_col = []
  cost_col = []
  status_col = []
  account_number_col = []
  account_name_col = []
  bvn_col = []
  requestSuccessful_col = []
  responseMessage_col = []
  responseCode_col = []


  list_column_none_api_request = []
  list_column_none_api_request.append(message_sms_payload_col)
  list_column_none_api_request.append(totalsent_col)
  list_column_none_api_request.append(cost_col)
  list_column_none_api_request.append(status_col)
  list_column_none_api_request.append(account_number_col)
  list_column_none_api_request.append(bvn_col)
  list_column_none_api_request.append(requestSuccessful_col)
  list_column_none_api_request.append(responseMessage_col)
  list_column_none_api_request.append(responseCode_col)
  list_column_none_api_request.append(account_name_col)
  
  list_column_none_client_mobile = []
  list_column_none_client_mobile.append(message_sms_payload_col)
  list_column_none_client_mobile.append(totalsent_col)
  list_column_none_client_mobile.append(cost_col)
  list_column_none_client_mobile.append(status_col)
  list_column_none_client_mobile.append(account_number_col)
  list_column_none_client_mobile.append(bvn_col)
  list_column_none_client_mobile.append(requestSuccessful_col)
  list_column_none_client_mobile.append(responseMessage_col)
  list_column_none_client_mobile.append(responseCode_col)
  list_column_none_client_mobile.append(account_name_col)
  list_column_none_client_mobile.append(endpoint_Col)

  list_column_none_sms_payload = []
  list_column_none_sms_payload.append(totalsent_col)
  list_column_none_sms_payload.append(cost_col)
  list_column_none_sms_payload.append(status_col)
  list_column_none_sms_payload.append(account_number_col)
  list_column_none_sms_payload.append(bvn_col)
  list_column_none_sms_payload.append(requestSuccessful_col)
  list_column_none_sms_payload.append(responseMessage_col)
  list_column_none_sms_payload.append(responseCode_col)
  list_column_none_sms_payload.append(account_name_col)
  list_column_none_sms_payload.append(email_col)
  list_column_none_sms_payload.append(endpoint_Col)
  list_column_none_sms_payload.append(date_col)

  list_column_none_sms_success = []
  list_column_none_sms_success.append(message_sms_payload_col)
  list_column_none_sms_success.append(account_number_col)
  list_column_none_sms_success.append(bvn_col)
  list_column_none_sms_success.append(requestSuccessful_col)
  list_column_none_sms_success.append(responseMessage_col)
  list_column_none_sms_success.append(responseCode_col)
  list_column_none_sms_success.append(account_name_col)
  list_column_none_sms_success.append(email_col)
  list_column_none_sms_success.append(phone_Col)
  list_column_none_sms_success.append(endpoint_Col)
  list_column_none_sms_success.append(date_col)


  list_column_none_wallet_success = []
  list_column_none_wallet_success.append(totalsent_col)
  list_column_none_wallet_success.append(message_sms_payload_col)
  list_column_none_wallet_success.append(cost_col)
  list_column_none_wallet_success.append(status_col)
  list_column_none_wallet_success.append(email_col)
  list_column_none_wallet_success.append(phone_Col)
  list_column_none_wallet_success.append(endpoint_Col)
  list_column_none_wallet_success.append(date_col)


  list_column_none_leadway_success = []
  list_column_none_leadway_success.append(message_sms_payload_col)
  list_column_none_leadway_success.append(totalsent_col)
  list_column_none_leadway_success.append(cost_col)
  list_column_none_leadway_success.append(status_col)
  list_column_none_leadway_success.append(account_number_col)
  list_column_none_leadway_success.append(bvn_col)
  list_column_none_leadway_success.append(requestSuccessful_col)
  list_column_none_leadway_success.append(responseMessage_col)
  list_column_none_leadway_success.append(responseCode_col)
  list_column_none_leadway_success.append(account_name_col)
  list_column_none_leadway_success.append(email_col)
  list_column_none_leadway_success.append(phone_Col)
  list_column_none_leadway_success.append(endpoint_Col)
  list_column_none_leadway_success.append(date_col)

  
  
  list_all_colum = []
  list_all_colum.append(type_request_col)
  list_all_colum.append(phone_Col)
  list_all_colum.append(date_col)
  list_all_colum.append(endpoint_Col)
  list_all_colum.append(log_level_col)
  list_all_colum.append(email_col)
  list_all_colum.append(message_sms_payload_col)
  list_all_colum.append(totalsent_col)
  list_all_colum.append(cost_col)
  list_all_colum.append(status_col)
  list_all_colum.append(account_number_col)
  list_all_colum.append(account_name_col)
  list_all_colum.append(bvn_col)
  list_all_colum.append(requestSuccessful_col)
  list_all_colum.append(responseMessage_col)
  list_all_colum.append(responseCode_col)

  for index, row in df_okra.iterrows():
    str_text = row['text']
    
    if not str_text.startswith('['):
      for i in range(len(list_all_colum)):
          list_all_colum[i].append(None)

    # check if the row contains "info" string
    if re.search('info', str_text):
        log_level = re.search('info', str_text)
        try:
          log_level_col.append(log_level.group(0))
        except AttributeError:
          log_level_col.append(None)             
        if 'mailto' not in str_text:
            if re.search('OKRA WEBHOOK', str_text):
                  type_of_request = re.search('OKRA WEBHOOK', str_text)
                  okra_webhook= parse_wallet_sms_payload_success(str_text)
                  try:
                    type_request_col.append(type_of_request.group(0))
                  except AttributeError:
                    type_request_col.append(None)
                  
                  for n in range(len(list_column_none_wallet_success)):
                    list_column_none_wallet_success[n].append(None)

                     
        elif re.search('OKRA PAYLOAD', str_text): # Nothing
          type_of_request = re.search('OKRA PAYLOAD', str_text)
        elif re.search('OKRA SUCCESS', str_text):   # Nothing
          type_of_request = re.search('OKRA SUCCESS', str_text)
        elif re.search('VTPASS SUCCESS', str_text):   # Nothing
          type_of_request = re.search('VTPASS SUCCESS', str_text)  

  #for x in range(len(list_all_colum)):
   # print(len(list_all_colum[x]))
  

  df_okra['Type_Request'] = type_request_col
  df_okra['Phone_Number'] = phone_Col
  df_okra['Date'] = date_col
  df_okra['EndPoint'] = endpoint_Col
  df_okra['Log_Level'] = log_level_col
  df_okra['Email'] = email_col
  df_okra['Message SMS Payload'] = message_sms_payload_col
  df_okra['Total Sent'] = totalsent_col
  df_okra['Cost'] = cost_col
  df_okra['Status'] = status_col
  df_okra['Account Number'] = account_number_col
  df_okra['Account Name'] = account_name_col
  df_okra['BVN'] = bvn_col
  df_okra['Request Successful'] = requestSuccessful_col
  df_okra['Response Message'] = responseMessage_col
  df_okra['Response Code'] = responseCode_col
 
  return df_okra


### Handle LEADWAY SUCCESS Type request

In [86]:
import re
import regex
import json

def parse_row_leadway_function(df_leadway_success):
  log_level_col = []
  api_request_col = []
  type_request_col = []
  phone_Col = []
  date_col = []
  endpoint_Col = []
  email_col = []
  message_sms_payload_col = []
  totalsent_col = []
  cost_col = []
  status_col = []
  account_number_col = []
  account_name_col = []
  bvn_col = []
  requestSuccessful_col = []
  responseMessage_col = []
  responseCode_col = []


  list_column_none_api_request = []
  list_column_none_api_request.append(message_sms_payload_col)
  list_column_none_api_request.append(totalsent_col)
  list_column_none_api_request.append(cost_col)
  list_column_none_api_request.append(status_col)
  list_column_none_api_request.append(account_number_col)
  list_column_none_api_request.append(bvn_col)
  list_column_none_api_request.append(requestSuccessful_col)
  list_column_none_api_request.append(responseMessage_col)
  list_column_none_api_request.append(responseCode_col)
  list_column_none_api_request.append(account_name_col)
  
  list_column_none_client_mobile = []
  list_column_none_client_mobile.append(message_sms_payload_col)
  list_column_none_client_mobile.append(totalsent_col)
  list_column_none_client_mobile.append(cost_col)
  list_column_none_client_mobile.append(status_col)
  list_column_none_client_mobile.append(account_number_col)
  list_column_none_client_mobile.append(bvn_col)
  list_column_none_client_mobile.append(requestSuccessful_col)
  list_column_none_client_mobile.append(responseMessage_col)
  list_column_none_client_mobile.append(responseCode_col)
  list_column_none_client_mobile.append(account_name_col)
  list_column_none_client_mobile.append(endpoint_Col)

  list_column_none_sms_payload = []
  list_column_none_sms_payload.append(totalsent_col)
  list_column_none_sms_payload.append(cost_col)
  list_column_none_sms_payload.append(status_col)
  list_column_none_sms_payload.append(account_number_col)
  list_column_none_sms_payload.append(bvn_col)
  list_column_none_sms_payload.append(requestSuccessful_col)
  list_column_none_sms_payload.append(responseMessage_col)
  list_column_none_sms_payload.append(responseCode_col)
  list_column_none_sms_payload.append(account_name_col)
  list_column_none_sms_payload.append(email_col)
  list_column_none_sms_payload.append(endpoint_Col)
  list_column_none_sms_payload.append(date_col)

  list_column_none_sms_success = []
  list_column_none_sms_success.append(message_sms_payload_col)
  list_column_none_sms_success.append(account_number_col)
  list_column_none_sms_success.append(bvn_col)
  list_column_none_sms_success.append(requestSuccessful_col)
  list_column_none_sms_success.append(responseMessage_col)
  list_column_none_sms_success.append(responseCode_col)
  list_column_none_sms_success.append(account_name_col)
  list_column_none_sms_success.append(email_col)
  list_column_none_sms_success.append(phone_Col)
  list_column_none_sms_success.append(endpoint_Col)
  list_column_none_sms_success.append(date_col)


  list_column_none_wallet_success = []
  list_column_none_wallet_success.append(totalsent_col)
  list_column_none_wallet_success.append(message_sms_payload_col)
  list_column_none_wallet_success.append(cost_col)
  list_column_none_wallet_success.append(status_col)
  list_column_none_wallet_success.append(email_col)
  list_column_none_wallet_success.append(phone_Col)
  list_column_none_wallet_success.append(endpoint_Col)
  list_column_none_wallet_success.append(date_col)


  list_column_none_leadway_success = []
  list_column_none_leadway_success.append(message_sms_payload_col)
  list_column_none_leadway_success.append(totalsent_col)
  list_column_none_leadway_success.append(cost_col)
  list_column_none_leadway_success.append(status_col)
  list_column_none_leadway_success.append(account_number_col)
  list_column_none_leadway_success.append(bvn_col)
  list_column_none_leadway_success.append(requestSuccessful_col)
  list_column_none_leadway_success.append(responseMessage_col)
  list_column_none_leadway_success.append(responseCode_col)
  list_column_none_leadway_success.append(account_name_col)
  list_column_none_leadway_success.append(email_col)
  list_column_none_leadway_success.append(phone_Col)
  list_column_none_leadway_success.append(endpoint_Col)
  list_column_none_leadway_success.append(date_col)

  
  
  list_all_colum = []
  list_all_colum.append(type_request_col)
  list_all_colum.append(phone_Col)
  list_all_colum.append(date_col)
  list_all_colum.append(endpoint_Col)
  list_all_colum.append(log_level_col)
  list_all_colum.append(email_col)
  list_all_colum.append(message_sms_payload_col)
  list_all_colum.append(totalsent_col)
  list_all_colum.append(cost_col)
  list_all_colum.append(status_col)
  list_all_colum.append(account_number_col)
  list_all_colum.append(account_name_col)
  list_all_colum.append(bvn_col)
  list_all_colum.append(requestSuccessful_col)
  list_all_colum.append(responseMessage_col)
  list_all_colum.append(responseCode_col)

  for index, row in df_leadway_success.iterrows():
    str_text = row['text']
    
    if not str_text.startswith('['):
      for i in range(len(list_all_colum)):
          list_all_colum[i].append(None)

    # check if the row contains "info" string
    if re.search('info', str_text):
        log_level = re.search('info', str_text)
        try:
          log_level_col.append(log_level.group(0))
        except AttributeError:
          log_level_col.append(None)             
        if 'mailto' not in str_text:
            if re.search('LEADWAY SUCCESS', str_text):
                  type_of_request = re.search('LEADWAY SUCCESS', str_text)
                  leadway_success_concat_text, index_first_succ, index_last_succ = parse_and_concatenate_Leadway_Success_Rows(df_)
                  #print(leadway_success_concat_text)
                  res_text_leadway = parse_Leadway_Success_Row(leadway_success_concat_text)
                  #print(res_text_leadway)
                  for o in range(len(list_column_none_leadway_success)):
                    list_column_none_leadway_success[o].append(None)
                  
                  #print('column none leadway success:')
                  #print(list_column_none_leadway_success)

                  try:
                    type_request_col.append(type_of_request.group(0))
                  except AttributeError:
                    type_request_col.append(None)

                     
        elif re.search('OKRA PAYLOAD', str_text): # Nothing
          type_of_request = re.search('OKRA PAYLOAD', str_text)
        elif re.search('OKRA SUCCESS', str_text):   # Nothing
          type_of_request = re.search('OKRA SUCCESS', str_text)
        elif re.search('VTPASS SUCCESS', str_text):   # Nothing
          type_of_request = re.search('VTPASS SUCCESS', str_text)  

  #for x in range(len(list_all_colum)):
   # print(len(list_all_colum[x]))
  

  df_leadway_success['Type_Request'] = type_request_col
  df_leadway_success['Phone_Number'] = phone_Col
  df_leadway_success['Date'] = date_col
  df_leadway_success['EndPoint'] = endpoint_Col
  df_leadway_success['Log_Level'] = log_level_col
  df_leadway_success['Email'] = email_col
  df_leadway_success['Message SMS Payload'] = message_sms_payload_col
  df_leadway_success['Total Sent'] = totalsent_col
  df_leadway_success['Cost'] = cost_col
  df_leadway_success['Status'] = status_col
  df_leadway_success['Account Number'] = account_number_col
  df_leadway_success['Account Name'] = account_name_col
  df_leadway_success['BVN'] = bvn_col
  df_leadway_success['Request Successful'] = requestSuccessful_col
  df_leadway_success['Response Message'] = responseMessage_col
  df_leadway_success['Response Code'] = responseCode_col
 
  return df_leadway_success
